In [6]:
import time, psutil, json, numpy as np
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Configuração Local

MILVUS_HOST = "localhost"  # ou "127.0.0.1"
MILVUS_PORT = "19540"      # Porta padrão do Milvus (verifique seu docker-compose)
COLLECTION_NAME = "lfw_cosface_modular"

def connect_to_milvus():
    try:
        # Tenta conectar. Se já existir, ele apenas confirma.
        connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
        print(f"✓ Conectado com sucesso ao Milvus em {MILVUS_HOST}:{MILVUS_PORT}")
    except Exception as e:
        print(f"✗ Erro ao conectar: {e}")

connect_to_milvus()

✓ Conectado com sucesso ao Milvus em localhost:19540


In [7]:
def setup_collection(name, dim, recreate=True):
    if utility.has_collection(name) and recreate:
        utility.drop_collection(name)
        print(f"✓ Coleção {name} removida para reinício.")

    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="person_id", dtype=DataType.VARCHAR, max_length=256), # Metadado
        FieldSchema(name="image_path", dtype=DataType.VARCHAR, max_length=512), # Metadado
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim)
    ]
    schema = CollectionSchema(fields, "Processamento Modular CosFace")
    return Collection(name, schema)

collection = setup_collection(COLLECTION_NAME, 1024)

In [10]:
import time, psutil, json, numpy as np

def run_individual_insertion(coll, embeddings, labels, limit=100):
    """MÉTODO A: Inserção Individual (Síncrona)"""
    print(f"-> Iniciando Ingestão Individual ({limit} vetores)...")
    t_start = time.time()
    
    for i in range(limit):
        # Inserindo um por um no formato de lista de listas
        coll.insert([
            [labels[i]], 
            [f"local_path/{labels[i]}.jpg"], 
            [embeddings[i].tolist()]
        ])
    
    total_time = time.time() - t_start
    print(f"⏱️ Tempo Método A: {total_time:.4f}s ({limit/total_time:.2f} vps)")
    return total_time

def run_bulk_insertion(coll, embeddings, labels, batch_size=1000):
    """MÉTODO B: Inserção em Lote (Bulk)"""
    num_total = len(embeddings)
    print(f"-> Iniciando Ingestão em Lote ({num_total} vetores)...")
    t_start = time.time()
    
    for i in range(0, num_total, batch_size):
        end = i + batch_size
        batch_labels = labels[i:end]
        batch_vectors = embeddings[i:end].tolist()
        # Gerando caminhos simulados para os metadados
        paths = [f"local_path/{label}.jpg" for label in batch_labels]
        
        coll.insert([batch_labels, paths, batch_vectors])
    
    coll.flush() # Força a persistência dos dados
    total_time = time.time() - t_start
    print(f"⏱️ Tempo Método B: {total_time:.4f}s ({num_total/total_time:.2f} vps)")
    return total_time

# --- EXECUÇÃO COM CAMINHOS ATUALIZADOS ---
try:
    # Atualizado de 'assets' para 'recursos' conforme sua mudança
    emb_data = np.load("recursos/lfw_resnet_embeddings.npy").astype('float32')
    with open("recursos/lfw_resnet_names.json", 'r') as f:
        label_data = json.load(f)

    # Executando os dois métodos para comparação
    time_a = run_individual_insertion(collection, emb_data, label_data, limit=100)
    time_b = run_bulk_insertion(collection, emb_data, label_data, batch_size=1000)

    # Cálculo de eficiência comparativa
    vps_a = 100 / time_a
    vps_b = len(emb_data) / time_b
    print(f"\n🏆 CONCLUSÃO: O Método Bulk é {vps_b / vps_a:.1f}x mais rápido localmente.")

except FileNotFoundError as e:
    print(f"❌ ERRO: Verifique se os arquivos estão na pasta 'recursos'. Detalhe: {e}")

-> Iniciando Ingestão Individual (100 vetores)...
⏱️ Tempo Método A: 2.8092s (35.60 vps)
-> Iniciando Ingestão em Lote (13233 vetores)...
⏱️ Tempo Método B: 72.9829s (181.32 vps)

🏆 CONCLUSÃO: O Método Bulk é 5.1x mais rápido localmente.
